# 베이스 코드(10%)

In [2]:
from sklearn.impute import KNNImputer

In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.05        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

KNN 보간
imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/3.A.R5M7C11'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order = 3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )


        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[INFO] Total values: 3,648,960  |  NaN count: 182,333  |  NaN ratio: 5.00%


KeyboardInterrupt: 

In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/1.A.M5C10'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[INFO] Pre-injection NaNs: 182,333 / 3,648,960 (5.00%)


KeyboardInterrupt: 

In [11]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00321804(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00321804(0002)/rack4_module1.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack16_module5.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module9.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack17_module10.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack15_module9.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack6_module8.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack4_module2.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module1.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack15_module17.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack13_module13.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack18_module14.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack10_module2.csv → A_5%/00321804(0002)
[OK] preprocessing_A/00321804(0002)/rack9_module17.csv → A_5%/00321804(0002)


In [12]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00329601(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00329601(0002)/rack4_module1.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module9.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack15_module9.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack6_module8.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack4_module2.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module1.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack15_module17.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack13_module13.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack10_module2.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack9_module17.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack10_module9.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack2_module17.csv → A_5%/00329601(0002)
[OK] preprocessing_A/00329601(0002)/rack11_module9.csv → A_5%/00329601(0002)
[O

In [13]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00391262(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00391262(0002)/rack24_module3.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack4_module1.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack16_module5.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack9_module9.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack20_module8.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack26_module1.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack17_module10.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack15_module9.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack6_module8.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack20_module15.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack22_module11.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack28_module5.csv → A_5%/00391262(0002)
[OK] preprocessing_A/00391262(0002)/rack4_module2.csv → A_5%/00391262(0002)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00451902(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00451902(0002)/rack4_module1.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack16_module5.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack9_module9.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack15_module9.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack6_module8.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack4_module2.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack9_module1.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack15_module17.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack13_module13.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack10_module2.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack9_module17.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack10_module9.csv → A_5%/00451902(0002)
[OK] preprocessing_A/00451902(0002)/rack2_module17.csv → A_5%/00451902(0002)
[O

# 여기부터 실행하기

In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00465382(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00465382(0002)/rack4_module1.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack16_module5.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack9_module9.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack15_module9.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack6_module8.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack4_module2.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack9_module1.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack15_module17.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack13_module13.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack10_module2.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack9_module17.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack10_module9.csv → A_5%/00465382(0002)
[OK] preprocessing_A/00465382(0002)/rack2_module17.csv → A_5%/00465382(0002)
[O

In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.15        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00466746(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_5%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


[OK] preprocessing_A/00466746(0002)/rack4_module1.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack16_module5.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack9_module9.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack17_module10.csv → A_5%/00466746(0002)


[OK] preprocessing_A/00466746(0002)/rack15_module9.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack6_module8.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack4_module2.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack9_module1.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack15_module17.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack13_module13.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack18_module14.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack10_module2.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack9_module17.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack10_module9.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack2_module17.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack16_module2.csv → A_5%/00466746(0002)
[OK] preprocessing_A/00466746(0002)/rack11_module9.csv → A_5%/00466746(0002)

# 00454547, 00370960

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.5      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_5%'  # 최상위 저장 폴더


def process_bank_data_2d(df, bank_label, base_filename, output_folder):
    """
    bank_no별 데이터를 받아서 베이스 코드 전처리를 적용한 후,
    채널별 2D (날짜수×1440) 배열로 저장합니다.
    """
    df_copy = df.copy()
    # 1. 시간 컬럼 처리
    if "colec_dt" not in df_copy.columns:
        if "clct_dt" in df_copy.columns:
            df_copy.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            print(f"{base_filename} - bank {bank_label}: 시간 컬럼 없음, 스킵")
            return

    # 2. 전압 컬럼 이름 정규화
    if not any(col.startswith("cell_volt_") for col in df_copy.columns):
        rename_dict = {}
        for col in df_copy.columns:
            if col.startswith("cel_volt_"):
                num = col.replace("cel_volt_", "")
                if num.isdigit():
                    rename_dict[col] = f"cell_volt_{int(num)}"
        if rename_dict:
            df_copy.rename(columns=rename_dict, inplace=True)

    # 3. datetime 변환 및 재인덱싱
    df_copy["colec_dt"] = pd.to_datetime(df_copy["colec_dt"])
    start, end = df_copy["colec_dt"].min(), df_copy["colec_dt"].max()
    full_idx = pd.DataFrame({"colec_dt": pd.date_range(start, end, freq='min')})
    merged = pd.merge(full_idx, df_copy, on="colec_dt", how="left")
    merged = merged.groupby("colec_dt").first().reset_index()

    # 4. 불필요 컬럼 제거
    if "Unnamed: 0" in merged.columns:
        merged.drop(columns=["Unnamed: 0"], inplace=True)

    # 5. 전압 NaN 처리 및 필터링
    volt_cols = [c for c in merged.columns if c.startswith("cell_volt_")]
    merged.loc[merged[volt_cols[0]] < 3.3, volt_cols] = np.nan
    merged["date"] = merged["colec_dt"].dt.date
    nan_thr = 1400
    bad = merged.groupby("date")[volt_cols[0]].apply(lambda x: x.isna().sum())
    ok_days = bad[bad <= nan_thr].index
    merged = merged[merged["date"].isin(ok_days)].reset_index(drop=True)

    # 6. ★ 결측치 5% 무작위 주입 ★
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged[volt_cols].shape) < MISSING_RATE
    merged.loc[:, volt_cols] = merged[volt_cols].mask(mask)

    # # 7. 선형 보간
    # merged[volt_cols] = merged[volt_cols].interpolate(method="polynomial", order=3)
    
    # 7. KNN 보간
    imputer = KNNImputer(n_neighbors=5,
                        weights='uniform',
                        metric='nan_euclidean')
    for day, idx in merged.groupby('date').groups.items():
        sub = merged.loc[idx, volt_cols]              # 해당 날짜(1,440행)만 추출
        imputed = imputer.fit_transform(sub)          # 하루치만 이웃 탐색
        merged.loc[idx, volt_cols] = pd.DataFrame(    # 보간 결과를 원위치에 덮어쓰기
            imputed,
            columns=volt_cols,
            index=idx
        )


    # 8. 유효일 필터링
    vt_thr, req_count = 3.6, 400*len(volt_cols)
    vc = merged.groupby("date")[volt_cols].apply(lambda df: (df>=vt_thr).sum().sum())
    keep = vc[vc>=req_count].index
    merged = merged[merged["date"].isin(keep)].reset_index(drop=True)

    # 9. 가장 앞/뒤 하루 제거
    dates = merged["date"].unique()
    if len(dates) <= 2:
        print(f"{base_filename} - bank {bank_label}: 유효일 부족")
        return
    merged = merged[~merged["date"].isin([dates[0], dates[-1]])].reset_index(drop=True)
    unique_dates = merged["date"].unique()

    # 10. 채널별 2D 배열 생성 및 저장 (float32)
    for volt in volt_cols:
        X = np.empty([len(unique_dates), 1440], dtype=np.float32)
        valid_idx = []
        for i, d in enumerate(unique_dates):
            day = merged[merged["date"]==d]
            series = day[volt]
            if series.shape[0] != 1440:
                print(f"{base_filename}, bank{bank_label}, {d}, {volt}: {series.shape[0]} rows")
                continue
            valid_idx.append(i)
            X[i] = series.to_numpy(dtype=np.float32)
        if not valid_idx:
            print(f"{base_filename} - bank {bank_label}: {volt} 유효 데이터 없음")
            continue
        X = X[valid_idx]
        out_path = os.path.join(output_folder, f"{base_filename}_{volt}.npy")
        np.save(out_path, X)
        print(f"Saved: {out_path} shape={X.shape}")

# ────────────────────────────────────────────────────────────────
# 사용 예시
# ────────────────────────────────────────────────────────────────
if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 고정

    input_folder = 'preprocessing_A/00454547(0002)'
    ess_files = glob.glob(os.path.join(input_folder, '*.csv'))
    base = os.path.basename(input_folder)

    # ★ A_5%/<base>_1, A_5%/<base>_2 폴더 생성 ★
    bank1_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_1")
    bank2_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_2")
    for p in [bank1_out, bank2_out]:
        os.makedirs(p, exist_ok=True)

    for path in ess_files:
        df = pd.read_csv(path)
        name = os.path.splitext(os.path.basename(path))[0]
        df1 = df[df['bsc_fg_no']==1]
        df2 = df[df['bsc_fg_no']==2]
        if not df1.empty:
            process_bank_data_2d(df1, 1, name, bank1_out)
        if not df2.empty:
            process_bank_data_2d(df2, 2, name, bank2_out)


Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_1.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_2.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_3.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_4.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_5.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_6.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_7.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_8.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_9.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_10.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_11.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_12.npy shape=(36, 1440)
Saved: A_5%/00454547(0002)_1/rack4_module1_cell_volt_13.npy shape=(36, 1440)
Saved: A

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.15      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_5%'  # 최상위 저장 폴더


def process_bank_data_2d(df, bank_label, base_filename, output_folder):
    """
    bank_no별 데이터를 받아서 베이스 코드 전처리를 적용한 후,
    채널별 2D (날짜수×1440) 배열로 저장합니다.
    """
    df_copy = df.copy()
    # 1. 시간 컬럼 처리
    if "colec_dt" not in df_copy.columns:
        if "clct_dt" in df_copy.columns:
            df_copy.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            print(f"{base_filename} - bank {bank_label}: 시간 컬럼 없음, 스킵")
            return

    # 2. 전압 컬럼 이름 정규화
    if not any(col.startswith("cell_volt_") for col in df_copy.columns):
        rename_dict = {}
        for col in df_copy.columns:
            if col.startswith("cel_volt_"):
                num = col.replace("cel_volt_", "")
                if num.isdigit():
                    rename_dict[col] = f"cell_volt_{int(num)}"
        if rename_dict:
            df_copy.rename(columns=rename_dict, inplace=True)

    # 3. datetime 변환 및 재인덱싱
    df_copy["colec_dt"] = pd.to_datetime(df_copy["colec_dt"])
    start, end = df_copy["colec_dt"].min(), df_copy["colec_dt"].max()
    full_idx = pd.DataFrame({"colec_dt": pd.date_range(start, end, freq='min')})
    merged = pd.merge(full_idx, df_copy, on="colec_dt", how="left")
    merged = merged.groupby("colec_dt").first().reset_index()

    # 4. 불필요 컬럼 제거
    if "Unnamed: 0" in merged.columns:
        merged.drop(columns=["Unnamed: 0"], inplace=True)

    # 5. 전압 NaN 처리 및 필터링
    volt_cols = [c for c in merged.columns if c.startswith("cell_volt_")]
    merged.loc[merged[volt_cols[0]] < 3.3, volt_cols] = np.nan
    merged["date"] = merged["colec_dt"].dt.date
    nan_thr = 1400
    bad = merged.groupby("date")[volt_cols[0]].apply(lambda x: x.isna().sum())
    ok_days = bad[bad <= nan_thr].index
    merged = merged[merged["date"].isin(ok_days)].reset_index(drop=True)

    # 6. ★ 결측치 5% 무작위 주입 ★
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged[volt_cols].shape) < MISSING_RATE
    merged.loc[:, volt_cols] = merged[volt_cols].mask(mask)

    # # 7. 선형 보간
    # merged[volt_cols] = merged[volt_cols].interpolate(method="polynomial", order=3)

    # 7. KNN 보간
    imputer = KNNImputer(n_neighbors=5,
                        weights='uniform',
                        metric='nan_euclidean')
    for day, idx in merged.groupby('date').groups.items():
        sub = merged.loc[idx, volt_cols]              # 해당 날짜(1,440행)만 추출
        imputed = imputer.fit_transform(sub)          # 하루치만 이웃 탐색
        merged.loc[idx, volt_cols] = pd.DataFrame(    # 보간 결과를 원위치에 덮어쓰기
            imputed,
            columns=volt_cols,
            index=idx
        )

    # 8. 유효일 필터링
    vt_thr, req_count = 3.6, 400*len(volt_cols)
    vc = merged.groupby("date")[volt_cols].apply(lambda df: (df>=vt_thr).sum().sum())
    keep = vc[vc>=req_count].index
    merged = merged[merged["date"].isin(keep)].reset_index(drop=True)

    # 9. 가장 앞/뒤 하루 제거
    dates = merged["date"].unique()
    if len(dates) <= 2:
        print(f"{base_filename} - bank {bank_label}: 유효일 부족")
        return
    merged = merged[~merged["date"].isin([dates[0], dates[-1]])].reset_index(drop=True)
    unique_dates = merged["date"].unique()

    # 10. 채널별 2D 배열 생성 및 저장 (float32)
    for volt in volt_cols:
        X = np.empty([len(unique_dates), 1440], dtype=np.float32)
        valid_idx = []
        for i, d in enumerate(unique_dates):
            day = merged[merged["date"]==d]
            series = day[volt]
            if series.shape[0] != 1440:
                print(f"{base_filename}, bank{bank_label}, {d}, {volt}: {series.shape[0]} rows")
                continue
            valid_idx.append(i)
            X[i] = series.to_numpy(dtype=np.float32)
        if not valid_idx:
            print(f"{base_filename} - bank {bank_label}: {volt} 유효 데이터 없음")
            continue
        X = X[valid_idx]
        out_path = os.path.join(output_folder, f"{base_filename}_{volt}.npy")
        np.save(out_path, X)
        print(f"Saved: {out_path} shape={X.shape}")

# ────────────────────────────────────────────────────────────────
# 사용 예시
# ────────────────────────────────────────────────────────────────
if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 고정

    input_folder = 'preprocessing_A/00370960(0002)'
    ess_files = glob.glob(os.path.join(input_folder, '*.csv'))
    base = os.path.basename(input_folder)

    # ★ A_5%/<base>_1, A_5%/<base>_2 폴더 생성 ★
    bank1_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_1")
    bank2_out = os.path.join(ROOT_OUTPUT_DIR, f"{base}_2")
    for p in [bank1_out, bank2_out]:
        os.makedirs(p, exist_ok=True)

    for path in ess_files:
        df = pd.read_csv(path)
        name = os.path.splitext(os.path.basename(path))[0]
        df1 = df[df['bsc_fg_no']==1]
        df2 = df[df['bsc_fg_no']==2]
        if not df1.empty:
            process_bank_data_2d(df1, 1, name, bank1_out)
        if not df2.empty:
            process_bank_data_2d(df2, 2, name, bank2_out)


Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_1.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_2.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_3.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_4.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_5.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_6.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_7.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_8.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_9.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_10.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_11.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_12.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_1/rack4_module1_cell_volt_13.npy shape=(34, 1440)
Saved: A

Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_1.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_2.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_3.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_4.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_5.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_6.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_7.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_8.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_9.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_10.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_11.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_12.npy shape=(34, 1440)
Saved: A_5%/00370960(0002)_2/rack4_module1_cell_volt_13.npy shape=(34, 1440)
Saved: A

# 2.A.R7M7C11

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────
# 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED = 42                # 고정 시드
MISSING_RATE = 0.05      # 결측치 비율 (5%)
ROOT_OUTPUT_DIR = 'A_5%'  # 최상위 저장 폴더

# 입력 폴더 및 CSV 파일 리스트
ess_name = 'preprocessing_A/2.A.R7M17C6'
ess_folder = os.path.basename(ess_name.rstrip('/'))
ess_list = glob.glob(os.path.join(ess_name, '*.csv'))

# ★ 출력 루트 폴더 생성: A_5%/<ess_folder> ★
output_root = os.path.join(ROOT_OUTPUT_DIR, ess_folder)
os.makedirs(output_root, exist_ok=True)

for ess_path in ess_list:
    # CSV 파일 읽기
    df = pd.read_csv(ess_path)
    
    # colec_dt 컬럼 처리: 없으면 clct_dt -> colec_dt
    if 'colec_dt' not in df.columns:
        if 'clct_dt' in df.columns:
            df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            continue

    # cell_volt 컬럼명 정규화
    if not any(col.startswith('cell_volt_') for col in df.columns):
        rename_dict = {}
        for col in df.columns:
            if col.startswith('cel_volt_'):
                num_str = col.replace('cel_volt_', '')
                if num_str.isdigit():
                    rename_dict[col] = f'cell_volt_{int(num_str)}'
        if rename_dict:
            df.rename(columns=rename_dict, inplace=True)
    
    # datetime 변환 및 중복 제거
    df['colec_dt'] = pd.to_datetime(df['colec_dt'])
    df = df.drop_duplicates(subset='colec_dt', keep='first')
    
    # 분 단위 타임라인 생성 후 merge
    start_time = df['colec_dt'].min()
    end_time   = df['colec_dt'].max()
    time_idx   = pd.DataFrame({'colec_dt': pd.date_range(start_time, end_time, freq='min')})
    merged_df  = pd.merge(time_idx, df, on='colec_dt', how='left')
    
    # 불필요 컬럼 제거
    if 'Unnamed: 0' in merged_df.columns:
        merged_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    # 전압 값 필터링 (3.3V 미만 -> NaN)
    cel_volt_cols = [c for c in merged_df.columns if c.startswith('cell_volt_')]
    merged_df.loc[merged_df['cell_volt_1'] < 3.3, cel_volt_cols] = np.nan
    
    # 날짜별 NaN 개수로 과도일 제거
    merged_df['date'] = merged_df['colec_dt'].dt.date
    nan_threshold = 1400
    nan_counts = merged_df.groupby('date')['cell_volt_1'].apply(lambda x: x.isna().sum())
    drop_dates = nan_counts[nan_counts > nan_threshold].index
    merged_nan_remove_df = merged_df[~merged_df['date'].isin(drop_dates)].reset_index(drop=True)
    
    # ★ 결측치 5% 무작위 주입 ★
    rng = np.random.default_rng(SEED)
    mask = rng.random(merged_nan_remove_df[cel_volt_cols].shape) < MISSING_RATE
    merged_nan_remove_df.loc[:, cel_volt_cols] = merged_nan_remove_df[cel_volt_cols].mask(mask)
    
    # # ★ 선형 보간 ★
    # merged_nan_remove_df[cel_volt_cols] = merged_nan_remove_df[cel_volt_cols].interpolate(method='polynomial', order=3)
    
    
    # ★ KNN 보간 ★
    imputer = KNNImputer(
        n_neighbors=5,
        weights='uniform',
        metric='nan_euclidean'
    )
    for day, idx in merged_nan_remove_df.groupby('date').groups.items():
        # 해당 날짜 그룹에서 적어도 하나의 관측값이 있는 컬럼만 선택
        cols = [c for c in cel_volt_cols if merged_nan_remove_df.loc[idx, c].notna().any()]
        if not cols:
            continue

        sub = merged_nan_remove_df.loc[idx, cols]
        imputed = imputer.fit_transform(sub)
        merged_nan_remove_df.loc[idx, cols] = pd.DataFrame(
            imputed,
            columns=cols,
            index=idx
        )

    # 전압 기준 유효 데이터 필터링
    voltage_threshold = 3.6
    required_valid_count = 400 * len(cel_volt_cols)
    valid_counts = merged_nan_remove_df.groupby('date')[cel_volt_cols] \
                            .apply(lambda x: (x >= voltage_threshold).sum().sum())
    keep_dates = valid_counts[valid_counts >= required_valid_count].index
    merged_valid_volt_df = merged_nan_remove_df[merged_nan_remove_df['date'].isin(keep_dates)] \
                                        .reset_index(drop=True)
    
    # 첫/마지막 날짜 제외
    if merged_valid_volt_df.empty:
        continue
    first_date = merged_valid_volt_df['date'].iloc[0]
    df_mid = merged_valid_volt_df[merged_valid_volt_df['date'] != first_date].reset_index(drop=True)
    last_date = df_mid['date'].iloc[-1]
    final_df = df_mid[df_mid['date'] != last_date].reset_index(drop=True)
    
    unique_dates = final_df['date'].unique()
    base_filename = os.path.splitext(os.path.basename(ess_path))[0]
    
    # 채널별 2D 배열 생성 및 저장 (float32)
    for volt in cel_volt_cols:
        X_array = np.empty([len(unique_dates), 1440], dtype=np.float32)
        for i, d in enumerate(unique_dates):
            series = final_df[final_df['date'] == d][volt]
            X_array[i] = series.to_numpy(dtype=np.float32)
        save_path = os.path.join(output_root, f"{base_filename}_{volt}.npy")
        np.save(save_path, X_array)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

# KNN 보간
imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/3.A.R5M7C11'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order = 3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )


        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/1.A.M5C10'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00321804(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00329601(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00391262(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)


In [ ]:
"""
ESS CSV 전처리 스크립트 (결측치 5 % 주입 + 선형 보간 + float32 저장)
- random seed = 42 로 고정해 재현성 확보
- 결측치는 선형 보간 직전에 cel_volt_* 컬럼에 5 % 무작위 주입
- 선형 보간(`method="linear"`)으로 결측치 채우기
- 결과 `.npy` 는 float32 타입으로 저장
- 저장 경로 구조
  A_5%/
    └─ <원본 ESS 폴더명>/
        └─ <basefilename>_cell_volt_<n>.npy

베이스 코드(사용자 제공)에서 요구사항만 반영하여 전체 재작성
"""

import os
import glob
import numpy as np
import pandas as pd
from typing import List

# ────────────────────────────────────────────────────────────────
# 0. 설정 파라미터
# ────────────────────────────────────────────────────────────────
SEED: int            = 42          # 고정 시드
MISSING_RATE: float  = 0.20        # 5 % 결측치 비율
VOLT_DROP_THR: float = 3.3         # 3.3 V 미만 → NaN
NAN_DAY_THR: int     = 1400        # 하루 NaN 허용 개수 상한
VALID_VOLT_THR: float = 3.6        # 3.6 V 이상 유효 전압 기준
VALID_COUNT_REQ: int = 400         # 날짜별(분*채널) 유효 샘플 수

imputer = KNNImputer(n_neighbors=5,
                    weights='uniform',
                    metric='nan_euclidean')

# 입력 ESS 폴더 (예시)
ESS_SRC_DIR = 'preprocessing_A/00451902(0002)'

# 출력 루트 폴더 (결측 5 %)
ROOT_OUTPUT_DIR = 'A_20%'


def inject_random_missing(df: pd.DataFrame,
                           cols: List[str],
                           rate: float,
                           seed: int = 42) -> pd.DataFrame:

    rng = np.random.default_rng(seed)
    mask = rng.random(df[cols].shape) < rate
    df_missing = df.copy()
    df_missing.loc[:, cols] = df_missing[cols].mask(mask)
    return df_missing


def preprocess_ess_folder(ess_dir: str, root_out_dir: str):
    """단일 ESS 폴더(여러 CSV) 전처리 후 .npy 저장"""

    csv_list = glob.glob(os.path.join(ess_dir, '*.csv'))
    if not csv_list:
        print(f'[WARN] No CSV found in {ess_dir}')
        return

    # ▶ 출력 폴더 생성: A_5%/<ESS폴더명>
    ess_folder_name = os.path.basename(ess_dir.rstrip('/'))
    output_dir = os.path.join(root_out_dir, ess_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    rng = np.random.default_rng(SEED)  # 폴더 단위 난수 생성기 (선택적)

    for csv_path in csv_list:
        df = pd.read_csv(csv_path)

        # 날짜 컬럼 통일
        if 'colec_dt' not in df.columns:
            if 'clct_dt' in df.columns:
                df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
            else:
                print(f'[SKIP] {csv_path} : no date column')
                continue

        # cell_volt 컬럼명 정규화
        if not any(col.startswith('cell_volt_') for col in df.columns):
            rename_map = {}
            for col in df.columns:
                if col.startswith('cel_volt_'):
                    try:
                        num = int(col.replace('cel_volt_', ''))
                        rename_map[col] = f'cell_volt_{num}'
                    except ValueError:
                        continue
            if rename_map:
                df.rename(columns=rename_map, inplace=True)

        # ── 2‑2. 시계열 보완 & 기본 클린업 ────────────────────────
        df['colec_dt'] = pd.to_datetime(df['colec_dt'])
        start, end = df['colec_dt'].min(), df['colec_dt'].max()
        full_timeline = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
        merged = pd.merge(full_timeline, df, on='colec_dt', how='left')

        if 'Unnamed: 0' in merged.columns:
            merged.drop(columns='Unnamed: 0', inplace=True)

        cel_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

        # 3. 전압 < 3.3V → NaN
        merged.loc[merged['cell_volt_1'] < VOLT_DROP_THR, cel_cols] = np.nan

        # 4. 하루 단위 NaN 과도일 제거
        merged['date'] = merged['colec_dt'].dt.date
        excessive_nan_dates = (
            merged.groupby('date')['cell_volt_1']
            .apply(lambda s: s.isna().sum())
            .loc[lambda s: s > NAN_DAY_THR]
            .index
        )
        clean_df = merged[~merged['date'].isin(excessive_nan_dates)].reset_index(drop=True)

        # ── 5. ★ 결측치 생성 (cel_cols 대상) ★ ───────────────
        clean_df = inject_random_missing(clean_df, cel_cols, MISSING_RATE, SEED)

        # # 6. 선형 보간
        # clean_df[cel_cols] = clean_df[cel_cols].interpolate(method='polynomial', order=3)

        # 날짜별로 그룹 나누어 fit_transform
        for day, idx in clean_df.groupby('date').groups.items():
            sub = clean_df.loc[idx, cel_cols]               # 해당 날짜(그룹)의 1,440행
            imputed = imputer.fit_transform(sub)            # 하루치만 거리 계산
            clean_df.loc[idx, cel_cols] = pd.DataFrame(      # 원위치에 덮어쓰기
                imputed,
                columns=cel_cols,
                index=idx
            )

        # 7. 전압 기준 유효 날짜 필터
        valid_counts = (
            clean_df.groupby('date')[cel_cols]
            .apply(lambda x: (x >= VALID_VOLT_THR).sum().sum())
        )
        keep_dates = valid_counts.loc[lambda s: s >= VALID_COUNT_REQ * len(cel_cols)].index
        valid_df = clean_df[clean_df['date'].isin(keep_dates)].reset_index(drop=True)

        # 8. 첫날/마지막날 제거
        if valid_df.empty:
            print(f'[WARN] {csv_path} : no valid dates left')
            continue
        first_date, last_date = valid_df['date'].iloc[0], valid_df['date'].iloc[-1]
        final_df = valid_df[(valid_df['date'] != first_date) & (valid_df['date'] != last_date)].reset_index(drop=True)
        if final_df.empty:
            print(f'[WARN] {csv_path} : nothing after dropping edge dates')
            continue

        # ── 9. 채널별 (날짜, 1440) float32 배열 저장 ────────────
        unique_dates = final_df['date'].unique()
        base_name = os.path.splitext(os.path.basename(csv_path))[0]

        for vcol in cel_cols:
            arr = np.empty((len(unique_dates), 1440), dtype=np.float32)
            for i, d in enumerate(unique_dates):
                arr[i] = final_df.loc[final_df['date'] == d, vcol].to_numpy(dtype=np.float32)

            np.save(os.path.join(output_dir, f'{base_name}_{vcol}.npy'), arr)

        print(f'[OK] {csv_path} → {output_dir}')


if __name__ == '__main__':
    np.random.seed(SEED)  # 전역 시드 (scikit‑learn 호환용)

    os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
    preprocess_ess_folder(ESS_SRC_DIR, ROOT_OUTPUT_DIR)
